# Part 2 - Mapping Yelp Search Results

## Obective

- For this CodeAlong, we will be working with the Yelp API results from last class. 
- You will load in the .csv.gz of your yelp results and prepare the data for visualization.
- You will use Plotly Express to create an interactive map with all of the results.

## Tools You Will Use
- Part 1:
    - Yelp API:
        - Getting Started: 
            - https://www.yelp.com/developers/documentation/v3/get_started

    - `YelpAPI` python package
        -  "YelpAPI": https://github.com/gfairchild/yelpapi
- Part 2:

    - Plotly Express: https://plotly.com/python/getting-started/
        - With Mapbox API: https://www.mapbox.com/
        - `px.scatter_mapbox` [Documentation](https://plotly.com/python/scattermapbox/): 




### Applying Code From
- [Advanced Transformations with Pandas - Part 1](https://login.codingdojo.com/m/376/12529/88086)
- [Advanced Transformations with Pandas - Part 2](https://login.codingdojo.com/m/376/12529/88088)

### Goal

- We want to create a map with every restaurant plotted as a scatter plot with detailed information that appears when we hover over a business
- We will use plotly express's `px.scatter_mapbox` function to accomplish this.
    - https://plotly.com/python/scattermapbox/
    
    - We will need a Mapbox API token for some of the options:
        - https://studio.mapbox.com/
    

# Loading Data from Part 1

In [153]:
## Plotly is not included in your dojo-env
!pip install plotly

In [154]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import json

## importing plotly 
import plotly.express as px

In [155]:
## Load in csv.gz
df = pd.read_csv('Data/Seattle-pizza.csv.gz')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843 entries, 0 to 842
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             843 non-null    object 
 1   alias          843 non-null    object 
 2   name           843 non-null    object 
 3   image_url      831 non-null    object 
 4   is_closed      843 non-null    bool   
 5   url            843 non-null    object 
 6   review_count   843 non-null    int64  
 7   categories     843 non-null    object 
 8   rating         843 non-null    float64
 9   coordinates    843 non-null    object 
 10  transactions   843 non-null    object 
 11  location       843 non-null    object 
 12  phone          808 non-null    float64
 13  display_phone  808 non-null    object 
 14  distance       843 non-null    float64
 15  price          748 non-null    object 
dtypes: bool(1), float64(3), int64(1), object(11)
memory usage: 99.7+ KB


In [156]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 47.608127, 'longitude': -122.302435}",[],"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113,NaN
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.5976491915013, 'longitude': -1...",['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,86,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"{'latitude': 47.624577, 'longitude': -122.325577}","['delivery', 'pickup']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,"{'latitude': 47.6146934, 'longitude': -122.312...",['delivery'],"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,$$
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,"{'latitude': 47.615379179632, 'longitude': -12...",['delivery'],"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405,$


## Required Preprocessing 

- 1. We need to get the latitude and longitude for each business as separate columns.
- We also want to be able to show the restaurants:
    - name,
    - price range
    - address
    - and if they do delivery or takeout.

### Separating Latitude and Longitude

In [157]:
## use .apply pd.Series to convert a dict to columns
df['coordinates'].apply(pd.Series)

,0
0,"{'latitude': 47.608127, 'longitude': -122.302435}"
1,"{'latitude': 47.5976491915013, 'longitude': -1..."
2,"{'latitude': 47.624577, 'longitude': -122.325577}"
3,"{'latitude': 47.6146934, 'longitude': -122.312..."
4,"{'latitude': 47.615379179632, 'longitude': -12..."
...,...
838,"{'latitude': 47.67964, 'longitude': -122.2907}"
839,"{'latitude': 47.676215, 'longitude': -122.205084}"
840,"{'latitude': 47.6157033283809, 'longitude': -1..."
841,"{'latitude': 47.6140799195084, 'longitude': -1..."


- Why didn't that work???

In [158]:
## slice out a single test coordinate
test_coord = df.loc[4,'coordinates']
test_coord

"{'latitude': 47.615379179632, 'longitude': -122.32323073437}"

- Its not a dictionary anymore!!! WHAT??
    - CSV files cant store iterables (lists, dictionaries) so they get converted to strings.

### Fixing the String-Dictionaries

- The json module has another version of load and dump called `json.loads` and `json.dumps`
    - These are designed to process STRINGS instead of files. 
    
- If we use `json.loads` we can convert our string dictionary into an actual dictionary. 

In [159]:
## Use json.loads on the test coordinate
#json.loads(test_coord)

- JSON requires double quotes!

In [160]:
## replace single ' with " 
test_coord = test_coord.replace("'",'"')
test_coord

'{"latitude": 47.615379179632, "longitude": -122.32323073437}'

In [161]:
## Use json.loads on the test coordinate, again
type(json.loads(test_coord))

dict

In [162]:
type(test_coord)

str

### Now, how can we apply this same process to the entire column??

In [163]:
## replace ' with " (entire column)
#df['coordinates'] = df['coordinates'].str.replace("'",'"')
#df['coordinates'] = df['coordinates'].apply(json.loads)
## apply json.loads


In [164]:
# df['coordinates'] = df['coordinates'].apply(json.loads)
# df['coordinates']

In [165]:
#df['coordinates'].apply(pd.Series)

In [166]:
# Power use of LAMBDA 
lat_long = df['coordinates'].apply(lambda x : pd.Series(json.loads(x.replace("'",'"'))))


In [167]:
# Concatenate the 2 new columns and drop the original.
df = pd.concat([df.drop(columns=['coordinates']),lat_long],axis =1)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113,NaN,47.608127,-122.302435
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,86,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['delivery', 'pickup']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,zXasSoalA8ALUs_uiKt3fg,grateful-bread-seattle,Grateful Bread,https://s3-media1.fl.yelpcdn.com/bphoto/dw_z_D...,False,https://www.yelp.com/biz/grateful-bread-seattl...,261,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",3.5,['delivery'],"{'address1': '7001 35th Ave NE', 'address2': '...",1.206525e+10,(206) 525-3166,7417.526253,$,47.679640,-122.290700
839,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,187,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"['pickup', 'delivery']","{'address1': '147 Park Ln', 'address2': '', 'a...",1.425577e+10,(425) 576-5407,9765.353049,$$,47.676215,-122.205084
840,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,64,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],"{'address1': '600 Bellevue Way NE', 'address2'...",1.425638e+10,(425) 638-1000,7123.208902,$$,47.615703,-122.200735
841,3cLNDJ2vI29LWNvU7vVIuQ,capital-one-café-bellevue-2,Capital One Café,https://s3-media1.fl.yelpcdn.com/bphoto/-o6eAW...,False,https://www.yelp.com/biz/capital-one-caf%C3%A9...,37,"[{'alias': 'banks', 'title': 'Banks & Credit U...",4.0,[],"{'address1': '400 Bellevue Way NE', 'address2'...",1.425372e+10,(425) 372-0250,7071.129218,$,47.614080,-122.201363


In [168]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 843 entries, 0 to 842
Data columns (total 17 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             843 non-null    object 
 1   alias          843 non-null    object 
 2   name           843 non-null    object 
 3   image_url      831 non-null    object 
 4   is_closed      843 non-null    bool   
 5   url            843 non-null    object 
 6   review_count   843 non-null    int64  
 7   categories     843 non-null    object 
 8   rating         843 non-null    float64
 9   transactions   843 non-null    object 
 10  location       843 non-null    object 
 11  phone          808 non-null    float64
 12  display_phone  808 non-null    object 
 13  distance       843 non-null    float64
 14  price          748 non-null    object 
 15  latitude       843 non-null    float64
 16  longitude      843 non-null    float64
dtypes: bool(1), float64(5), int64(1), object(10)
memory us

## Creating a Simple Map

### Register for MapBox API

Mapbox API: https://www.mapbox.com/

In [169]:
## Load in mapbox api credentials from .secret
with open('/Users/purvikansara/.secret/mapbox.json') as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

- Use the plotly express `set_maptbox_acccess_token` function

In [170]:
## set mapbox token
px.set_mapbox_access_token(login['api-key'])

In [171]:
## use scatter_mapbox for M.V.P map
px.scatter_mapbox(df,lat = 'latitude',lon = 'longitude',mapbox_style='open-street-map')

### Adding Hover Data

- We want to show the restaurants:
    - name
    - price range
    - address
    - and if they do delivery or takeout.
    
    
- We can use the `hover_name` and `hover_data` arguments for `px.scatter_mapbox` to add this info!

In [172]:
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{'address1': '2300 E Cherry St', 'address2': '...",NaN,NaN,749.173113,NaN,47.608127,-122.302435
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{'address1': '525 Rainier Ave S', 'address2': ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,86,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['delivery', 'pickup']","{'address1': '601 Summit Ave E', 'address2': '...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{'address1': '1546 15th Ave', 'address2': '', ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{'address1': '700 E Pine St', 'address2': '', ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231


In [173]:
## add hover_name (name) and hover_data for price,rating,location
px.scatter_mapbox(df,lat = 'latitude',lon = 'longitude',mapbox_style='open-street-map',hover_name='name',hover_data=['price','location','transactions'])

In [174]:
df['location']

0      {'address1': '2300 E Cherry St', 'address2': '...
1      {'address1': '525 Rainier Ave S', 'address2': ...
2      {'address1': '601 Summit Ave E', 'address2': '...
3      {'address1': '1546 15th Ave', 'address2': '', ...
4      {'address1': '700 E Pine St', 'address2': '', ...
                             ...                        
838    {'address1': '7001 35th Ave NE', 'address2': '...
839    {'address1': '147 Park Ln', 'address2': '', 'a...
840    {'address1': '600 Bellevue Way NE', 'address2'...
841    {'address1': '400 Bellevue Way NE', 'address2'...
842    {'address1': '1015 108th Ave NE', 'address2': ...
Name: location, Length: 843, dtype: object

### Fixing the Location Column

In [175]:
## slice out a test address
test_add = df.loc[0,'location']
type(test_add)

str

> Also a string-dictionary...

In [176]:
## replace ' with "
df['location'] = df['location'].str.replace("'", '"')
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,NaN,47.608127,-122.302435
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,86,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['delivery', 'pickup']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,zXasSoalA8ALUs_uiKt3fg,grateful-bread-seattle,Grateful Bread,https://s3-media1.fl.yelpcdn.com/bphoto/dw_z_D...,False,https://www.yelp.com/biz/grateful-bread-seattl...,261,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",3.5,['delivery'],"{""address1"": ""7001 35th Ave NE"", ""address2"": ""...",1.206525e+10,(206) 525-3166,7417.526253,$,47.679640,-122.290700
839,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,187,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"['pickup', 'delivery']","{""address1"": ""147 Park Ln"", ""address2"": """", ""a...",1.425577e+10,(425) 576-5407,9765.353049,$$,47.676215,-122.205084
840,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,64,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],"{""address1"": ""600 Bellevue Way NE"", ""address2""...",1.425638e+10,(425) 638-1000,7123.208902,$$,47.615703,-122.200735
841,3cLNDJ2vI29LWNvU7vVIuQ,capital-one-café-bellevue-2,Capital One Café,https://s3-media1.fl.yelpcdn.com/bphoto/-o6eAW...,False,https://www.yelp.com/biz/capital-one-caf%C3%A9...,37,"[{'alias': 'banks', 'title': 'Banks & Credit U...",4.0,[],"{""address1"": ""400 Bellevue Way NE"", ""address2""...",1.425372e+10,(425) 372-0250,7071.129218,$,47.614080,-122.201363


In [177]:
## apply json.loads
df['location'].apply(json.loads)

JSONDecodeError: Expecting value: line 1 column 62 (char 61)

> Ruh roh....

- Hmm, let's slice out a test_address again and let's write a function to accomplish this instead.
    - We can use try and except in our function to get around the errors.

### Fixing Addresses - with a custom function


In [179]:
## slice out test address 
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "2300 E Cherry St", "address2": "", "address3": None, "city": "Seattle", "zip_code": "98122", "country": "US", "state": "WA", "display_address": ["2300 E Cherry St", "Seattle, WA 98122"]}'

In [180]:
test_addr.split('address2')[0]
#test_addr[0]

'{"address1": "2300 E Cherry St", "'

In [181]:
## write a function to just run json.loads on the address
def fix_address(addr):
    try:
        return json.loads(addr)
    except:
        return "Error"
        


In [182]:
## test applying our function
df['location'].apply(fix_address)

0                                                  Error
1      {'address1': '525 Rainier Ave S', 'address2': ...
2                                                  Error
3      {'address1': '1546 15th Ave', 'address2': '', ...
4      {'address1': '700 E Pine St', 'address2': '', ...
                             ...                        
838    {'address1': '7001 35th Ave NE', 'address2': '...
839    {'address1': '147 Park Ln', 'address2': '', 'a...
840    {'address1': '600 Bellevue Way NE', 'address2'...
841                                                Error
842                                                Error
Name: location, Length: 843, dtype: object

- It worked! Now let's save this as a new column (display_location),
and then let's investigate the businesses that had an "ERROR".

In [183]:
### save a new display_location column using our function
df['display_loc'] = df['location'].apply(fix_address)
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_loc
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,NaN,47.608127,-122.302435,Error
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ..."
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,86,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['delivery', 'pickup']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577,Error
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ..."
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,zXasSoalA8ALUs_uiKt3fg,grateful-bread-seattle,Grateful Bread,https://s3-media1.fl.yelpcdn.com/bphoto/dw_z_D...,False,https://www.yelp.com/biz/grateful-bread-seattl...,261,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",3.5,['delivery'],"{""address1"": ""7001 35th Ave NE"", ""address2"": ""...",1.206525e+10,(206) 525-3166,7417.526253,$,47.679640,-122.290700,"{'address1': '7001 35th Ave NE', 'address2': '..."
839,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,187,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"['pickup', 'delivery']","{""address1"": ""147 Park Ln"", ""address2"": """", ""a...",1.425577e+10,(425) 576-5407,9765.353049,$$,47.676215,-122.205084,"{'address1': '147 Park Ln', 'address2': '', 'a..."
840,rcDbUcs83gvoRxhaTmFfnQ,cypress-lounge-and-wine-bar-bellevue,Cypress Lounge & Wine Bar,https://s3-media2.fl.yelpcdn.com/bphoto/DsFhIb...,False,https://www.yelp.com/biz/cypress-lounge-and-wi...,64,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...",3.5,[],"{""address1"": ""600 Bellevue Way NE"", ""address2""...",1.425638e+10,(425) 638-1000,7123.208902,$$,47.615703,-122.200735,"{'address1': '600 Bellevue Way NE', 'address2'..."
841,3cLNDJ2vI29LWNvU7vVIuQ,capital-one-café-bellevue-2,Capital One Café,https://s3-media1.fl.yelpcdn.com/bphoto/-o6eAW...,False,https://www.yelp.com/biz/capital-one-caf%C3%A9...,37,"[{'alias': 'banks', 'title': 'Banks & Credit U...",4.0,[],"{""address1"": ""400 Bellevue Way NE"", ""address2""...",1.425372e+10,(425) 372-0250,7071.129218,$,47.614080,-122.201363,Error


In [184]:
## filter for businesses with display_location == "ERROR"
df.loc[df['display_loc']=='Error']

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_loc
0,t-Z9bvmlgUyDtomGmttrUQ,guerilla-pizza-kitchen-seattle,Guerilla Pizza Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/uMrdEj...,False,https://www.yelp.com/biz/guerilla-pizza-kitche...,1,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,[],"{""address1"": ""2300 E Cherry St"", ""address2"": ""...",NaN,NaN,749.173113,NaN,47.608127,-122.302435,Error
2,EwrAoy7XbX8nfemZdDvupA,cornelly-seattle,Cornelly,https://s3-media4.fl.yelpcdn.com/bphoto/HlYYBT...,False,https://www.yelp.com/biz/cornelly-seattle?adju...,86,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.5,"['delivery', 'pickup']","{""address1"": ""601 Summit Ave E"", ""address2"": ""...",NaN,NaN,2578.701393,NaN,47.624577,-122.325577,Error
5,FVbwpNA1uZEGiM02N4XtUg,blotto-seattle,Blotto,https://s3-media3.fl.yelpcdn.com/bphoto/Q4Y1Cw...,False,https://www.yelp.com/biz/blotto-seattle?adjust...,30,"[{'alias': 'pizza', 'title': 'Pizza'}]",5.0,[],"{""address1"": ""1830 12th Ave"", ""address2"": """", ...",1.206403e+10,(206) 403-1809,1686.862176,NaN,47.618490,-122.316640,Error
6,ugTsEtjvwRhteac_6JcuPw,italian-family-pizza-seattle,Italian Family Pizza,https://s3-media2.fl.yelpcdn.com/bphoto/SgiSrJ...,False,https://www.yelp.com/biz/italian-family-pizza-...,972,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,"['delivery', 'pickup']","{""address1"": ""1028 Madison St"", ""address2"": No...",1.206538e+10,(206) 538-0040,2271.307105,$$,47.609370,-122.325460,Error
9,IrohtoYjnAR_vc6w6CRCxA,moto-seattle,Moto,https://s3-media2.fl.yelpcdn.com/bphoto/FBmvjy...,False,https://www.yelp.com/biz/moto-seattle?adjust_c...,110,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,['delivery'],"{""address1"": ""4526 42nd Ave SW"", ""address2"": ""...",1.206421e+10,(206) 420-8880,8784.075722,NaN,47.562070,-122.385090,Error
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
826,AsgtzyqpdjuPyjaEUVcUXA,jewel-of-india-seattle-3,Jewel Of India,https://s3-media2.fl.yelpcdn.com/bphoto/qJGMnl...,False,https://www.yelp.com/biz/jewel-of-india-seattl...,401,"[{'alias': 'indpak', 'title': 'Indian'}]",3.5,"['pickup', 'delivery']","{""address1"": ""4735 University Way NE"", ""addres...",1.206524e+10,(206) 523-5275,5825.461180,$$,47.664103,-122.313426,Error
830,nEZ8Xgkqj2OjQWw0qFpj7A,cherry-street-coffee-house-seattle-2,Cherry Street Coffee House,https://s3-media3.fl.yelpcdn.com/bphoto/vdgyWB...,False,https://www.yelp.com/biz/cherry-street-coffee-...,165,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",3.5,['delivery'],"{""address1"": ""2719 1st Ave"", ""address2"": None,...",1.206442e+10,(206) 441-5489,4206.008062,$,47.616190,-122.351608,Error
833,oniQ68t6VR1naVtUXmfETQ,pokeworks-bellevue-2,Pokeworks,https://s3-media1.fl.yelpcdn.com/bphoto/DqSOK8...,False,https://www.yelp.com/biz/pokeworks-bellevue-2?...,266,"[{'alias': 'poke', 'title': 'Poke'}, {'alias':...",4.0,"['pickup', 'delivery']","{""address1"": ""222 Bellevue Way NE"", ""address2""...",1.425214e+10,(425) 214-1182,7072.982860,$$,47.612521,-122.201333,Error
841,3cLNDJ2vI29LWNvU7vVIuQ,capital-one-café-bellevue-2,Capital One Café,https://s3-media1.fl.yelpcdn.com/bphoto/-o6eAW...,False,https://www.yelp.com/biz/capital-one-caf%C3%A9...,37,"[{'alias': 'banks', 'title': 'Banks & Credit U...",4.0,[],"{""address1"": ""400 Bellevue Way NE"", ""address2""...",1.425372e+10,(425) 372-0250,7071.129218,$,47.614080,-122.201363,Error


In [185]:
## slice out a new test address and inspect
test_addr = df.loc[0, 'location']
test_addr

'{"address1": "2300 E Cherry St", "address2": "", "address3": None, "city": "Seattle", "zip_code": "98122", "country": "US", "state": "WA", "display_address": ["2300 E Cherry St", "Seattle, WA 98122"]}'

> After some more investigation, we would find a few issues with these "ERROR" rows.
1. They contained None.
2. They contained an apostrophe in the name.
3. ...?

### Possible Fixes (if we care to/have the time)


- Use Regular Expressions to find an fix the display addresses with "'" in them
- Use string split to split on the word display address.
    - Then use string methods to clean up

### Moving Forward without those rows (for now)

In [186]:
## remove any rows where display_location == 'ERROR'
df = df.loc[df['display_loc']!='Error']
df

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_loc
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ..."
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ..."
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ..."
7,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,191,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['delivery', 'pickup']","{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,$$,47.598976,-122.294685,"{'address1': '2901 S Jackson St', 'address2': ..."
8,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,179,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,TJLPq5XM2gU3whWksAR5iw,kfc-seattle-8,KFC,https://s3-media4.fl.yelpcdn.com/bphoto/xeIb_l...,False,https://www.yelp.com/biz/kfc-seattle-8?adjust_...,53,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"['pickup', 'delivery']","{""address1"": ""1140 NW Market St"", ""address2"": ...",1.206789e+10,(206) 789-0282,8478.426534,$,47.668896,-122.372807,"{'address1': '1140 NW Market St', 'address2': ..."
837,tyjUpcq1IYfgrss-HO9hog,panera-bread-bellevue,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/XyC46x...,False,https://www.yelp.com/biz/panera-bread-bellevue...,218,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.5,"['pickup', 'delivery']","{""address1"": ""1100 Bellevue Way NE"", ""address2...",1.425453e+10,(425) 453-2600,7246.971006,$,47.620452,-122.199618,"{'address1': '1100 Bellevue Way NE', 'address2..."
838,zXasSoalA8ALUs_uiKt3fg,grateful-bread-seattle,Grateful Bread,https://s3-media1.fl.yelpcdn.com/bphoto/dw_z_D...,False,https://www.yelp.com/biz/grateful-bread-seattl...,261,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",3.5,['delivery'],"{""address1"": ""7001 35th Ave NE"", ""address2"": ""...",1.206525e+10,(206) 525-3166,7417.526253,$,47.679640,-122.290700,"{'address1': '7001 35th Ave NE', 'address2': '..."
839,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcdn.com/bphoto/tnvGkA...,False,https://www.yelp.com/biz/pho-tai-kirkland?adju...,187,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...",3.5,"['pickup', 'delivery']","{""address1"": ""147 Park Ln"", ""address2"": """", ""a...",1.425577e+10,(425) 576-5407,9765.353049,$$,47.676215,-122.205084,"{'address1': '147 Park Ln', 'address2': '', 'a..."


In [188]:
df.loc[840,'display_loc']

{'address1': '600 Bellevue Way NE',
 'address2': '',
 'address3': '',
 'city': 'Bellevue',
 'zip_code': '98004',
 'country': 'US',
 'state': 'WA',
 'display_address': ['600 Bellevue Way NE', 'Bellevue, WA 98004']}

- We want the "display_address" key from the "display_location" dictionaries.
- We could use a .apply and a lamda to slice out the desired key.

In [190]:
## use apply and lambda to slice correct key
df['display_address'] = df['display_loc'].apply(lambda x: x['display_address'])
df

/var/folders/td/cybzgn316hq8r5cn05lb0y2m0000gn/T/ipykernel_22627/1340834937.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_loc,display_address
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]"
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]"
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ...","[700 E Pine St, Seattle, WA 98122]"
7,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,191,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['delivery', 'pickup']","{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,$$,47.598976,-122.294685,"{'address1': '2901 S Jackson St', 'address2': ...","[2901 S Jackson St, Seattle, WA 98144]"
8,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,179,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ...","[4235 E Madison St, Seattle, WA 98112]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,TJLPq5XM2gU3whWksAR5iw,kfc-seattle-8,KFC,https://s3-media4.fl.yelpcdn.com/bphoto/xeIb_l...,False,https://www.yelp.com/biz/kfc-seattle-8?adjust_...,53,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"['pickup', 'delivery']","{""address1"": ""1140 NW Market St"", ""address2"": ...",1.206789e+10,(206) 789-0282,8478.426534,$,47.668896,-122.372807,"{'address1': '1140 NW Market St', 'address2': ...","[1140 NW Market St, Seattle, WA 98107]"
837,tyjUpcq1IYfgrss-HO9hog,panera-bread-bellevue,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/XyC46x...,False,https://www.yelp.com/biz/panera-bread-bellevue...,218,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.5,"['pickup', 'delivery']","{""address1"": ""1100 Bellevue Way NE"", ""address2...",1.425453e+10,(425) 453-2600,7246.971006,$,47.620452,-122.199618,"{'address1': '1100 Bellevue Way NE', 'address2...","[1100 Bellevue Way NE, Suite 9, Bellevue, WA 9..."
838,zXasSoalA8ALUs_uiKt3fg,grateful-bread-seattle,Grateful Bread,https://s3-media1.fl.yelpcdn.com/bphoto/dw_z_D...,False,https://www.yelp.com/biz/grateful-bread-seattl...,261,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",3.5,['delivery'],"{""address1"": ""7001 35th Ave NE"", ""address2"": ""...",1.206525e+10,(206) 525-3166,7417.526253,$,47.679640,-122.290700,"{'address1': '7001 35th Ave NE', 'address2': '...","[7001 35th Ave NE, Seattle, WA 98115]"
839,8S3qU0TGMJDyj4KHBmxlaw,pho-tai-kirkland,Pho Tai,https://s3-media3.fl.yelpcd

- Almost done! We want to convert display_address to a string instead a list of strings.
- We can use the string method .join to do so!

In [191]:
## slice out a test_address
test =df.loc[1,'display_address']
test

['525 Rainier Ave S', 'Seattle, WA 98144']

In [193]:
## test using .join with a "\n"
print("\n".join(test))

525 Rainier Ave S
Seattle, WA 98144


In [194]:
## apply the join to every row with a lambda
df['Address'] = df['display_address'].apply(lambda x: "\n".join(x))
df

/var/folders/td/cybzgn316hq8r5cn05lb0y2m0000gn/T/ipykernel_22627/3208991675.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,id,alias,name,image_url,is_closed,url,review_count,categories,rating,transactions,location,phone,display_phone,distance,price,latitude,longitude,display_loc,display_address,Address
1,Wi6LFkjIausYj277ru6pqg,humble-pie-seattle,Humble Pie,https://s3-media1.fl.yelpcdn.com/bphoto/9tK83_...,False,https://www.yelp.com/biz/humble-pie-seattle?ad...,349,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""525 Rainier Ave S"", ""address2"": ...",1.206330e+10,(206) 329-5133,2166.834011,$$,47.597649,-122.313306,"{'address1': '525 Rainier Ave S', 'address2': ...","[525 Rainier Ave S, Seattle, WA 98144]","525 Rainier Ave S\nSeattle, WA 98144"
3,fxyHWmfzcdjImgQ_UYHoTw,bar-cotto-seattle,Bar Cotto,https://s3-media3.fl.yelpcdn.com/bphoto/07DoPF...,False,https://www.yelp.com/biz/bar-cotto-seattle?adj...,330,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.5,['delivery'],"{""address1"": ""1546 15th Ave"", ""address2"": """", ...",1.206839e+10,(206) 838-8081,1292.526796,$$,47.614693,-122.312764,"{'address1': '1546 15th Ave', 'address2': '', ...","[1546 15th Ave, Seattle, WA 98122]","1546 15th Ave\nSeattle, WA 98122"
4,M9xzvwgK58T0w7wvXedvuQ,hot-mamas-pizza-seattle,Hot Mama's Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/LEL1qj...,False,https://www.yelp.com/biz/hot-mamas-pizza-seatt...,925,"[{'alias': 'pizza', 'title': 'Pizza'}]",4.0,['delivery'],"{""address1"": ""700 E Pine St"", ""address2"": """", ...",1.206323e+10,(206) 322-6444,2080.359405,$,47.615379,-122.323231,"{'address1': '700 E Pine St', 'address2': '', ...","[700 E Pine St, Seattle, WA 98122]","700 E Pine St\nSeattle, WA 98122"
7,et4eWK49x5od4lfqCukG6w,central-pizza-seattle,Central Pizza,https://s3-media4.fl.yelpcdn.com/bphoto/DE4G4e...,False,https://www.yelp.com/biz/central-pizza-seattle...,191,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...",4.0,"['delivery', 'pickup']","{""address1"": ""2901 S Jackson St"", ""address2"": ...",1.206603e+10,(206) 602-6333,1571.867438,$$,47.598976,-122.294685,"{'address1': '2901 S Jackson St', 'address2': ...","[2901 S Jackson St, Seattle, WA 98144]","2901 S Jackson St\nSeattle, WA 98144"
8,cAFXng3Gr1cBCFkP1VIxng,the-independent-pizzeria-seattle-2,The Independent Pizzeria,https://s3-media4.fl.yelpcdn.com/bphoto/4gIuis...,False,https://www.yelp.com/biz/the-independent-pizze...,179,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",4.0,['delivery'],"{""address1"": ""4235 E Madison St"", ""address2"": ...",1.206861e+10,(206) 860-6110,2897.252844,$$,47.636044,-122.277377,"{'address1': '4235 E Madison St', 'address2': ...","[4235 E Madison St, Seattle, WA 98112]","4235 E Madison St\nSeattle, WA 98112"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,TJLPq5XM2gU3whWksAR5iw,kfc-seattle-8,KFC,https://s3-media4.fl.yelpcdn.com/bphoto/xeIb_l...,False,https://www.yelp.com/biz/kfc-seattle-8?adjust_...,53,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",1.5,"['pickup', 'delivery']","{""address1"": ""1140 NW Market St"", ""address2"": ...",1.206789e+10,(206) 789-0282,8478.426534,$,47.668896,-122.372807,"{'address1': '1140 NW Market St', 'address2': ...","[1140 NW Market St, Seattle, WA 98107]","1140 NW Market St\nSeattle, WA 98107"
837,tyjUpcq1IYfgrss-HO9hog,panera-bread-bellevue,Panera Bread,https://s3-media2.fl.yelpcdn.com/bphoto/XyC46x...,False,https://www.yelp.com/biz/panera-bread-bellevue...,218,"[{'alias': 'sandwiches', 'title': 'Sandwiches'...",2.5,"['pickup', 'delivery']","{""address1"": ""1100 Bellevue Way NE"", ""address2...",1.425453e+10,(425) 453-2600,7246.971006,$,47.620452,-122.199618,"{'address1': '1100 Bellevue Way NE', 'address2...","[1100 Bellevue Way NE, Suite 9, Bellevue, WA 9...","1100 Bellevue Way NE\nSuite 9\nBellevue, WA 98004"
838,zXasSoalA8ALUs_uiKt3fg,grateful-bread-seattle,Grateful Bread,https://s3-media1.fl.yelpcdn.com/bphoto/dw_z_D...,False,https://www.yelp.com/biz/grateful-bread-seattl...,261,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",3.5,['delivery'],"{""

### Final Map

In [196]:
## make ourn final map and save as varaible
## add hover_name (name) and hover_data for price,rating,location
fig = px.scatter_mapbox(df,lat = 'latitude',lon = 'longitude',mapbox_style='open-street-map',
                  hover_name='name',hover_data=['price','Address','transactions'],color = 'rating')
fig

#### HTML Uses `<br>` instead of `\n`

In [109]:
## remake the final address column with <br> instead 

## plot the final map

In [197]:
## use fig.write_html to save map
fig.write_html('final_map.html')